## 1. Import libraries

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import backend as K
from keras.models import load_model

## 2. Loading model
Load the pre-trained **YOLO** model using Darknet. Also load the model's configuration and weights files.

In [ ]:
# Load the YOLO model
model = cv2.dnn.readNetFromDarknet('cfg/yolov3.cfg', 'model/yolov3.weights')

# Load the class names
classes = []
with open('model/coco.names', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

## 3. Load image
Load the image that we want to analyze

In [ ]:
# Load the image
img = cv2.imread("images/damaged_car.jpg")

# Convert the image to RGB format
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

## 4. Object  Detection
Next, we use the YOLO model to detect objects and any damage to them.

In [ ]:
# Resize the image
image_size = (416, 416)
img_resized = cv2.resize(img, image_size)

# Convert the image to a blob
blob = cv2.dnn.blobFromImage(img_resized, 1/255, image_size, [0, 0, 0], swapRB=True, crop=False)

# Set the input of the model
model.setInput(blob)

# Forward pass through the model
output_layers = model.getUnconnectedOutLayersNames()
outs = model.forward(output_layers)

# Extract the bounding boxes and classes
boxes = []
confidences = []
class_ids = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5 and class_id == 0:
            center_x = int(detection[0] * image_size[0])
            center_y = int(detection[1] * image_size[1])
            w = int(detection[2] * image_size[0])
            h = int(detection[3] * image_size[1])
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Perform non-maximum suppression
nms_threshold = 0.5
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, nms_threshold)

# Display the image with the detected objects
for i in indices:
    i = i[0]
    x, y, w, h = boxes[i]
    cv2.rectangle(img_resized, (x, y), (x + w, y + h), (0, 0, 255), 2)

plt.imshow(img_resized)
plt.show()

## 5. Damage Analysis
Once detected the damage to the object, analyze it by computing the area of the object by multiplying its width and height.

In [ ]:
# # Compute the area of the detected object
# x1, y1, x2, y2 = x, y, x + w, y + h
# w = x2 - x1
# h = y2 - y1
# area = w * h
# print('Area of damage:', area)








import cv2

# Load YOLOv3 model
net = cv2.dnn.readNet('cfg/yolov3.cfg', 'model/yolov3.weights')

# Load the image
image = cv2.imread('images/most-congested-cities.jpg')

# Get image dimensions
height, width, _ = image.shape

# Prepare the image for YOLO model
blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)

# Set the input to the neural network
net.setInput(blob)

# Get the names of the output layers
output_layers = net.getUnconnectedOutLayersNames()

# Forward pass to get the detected objects
detections = net.forward(output_layers)
# Define class labels
class_labels = [ 'person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
                 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
                 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 
                 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 
                 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 
                 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 
                 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 
                 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 
                 'teddy bear', 'hair drier', 'toothbrush'
]

# Set confidence threshold
confidence_threshold = 0.7

# Process detections
for detection in detections:
    for obj in detection:
        scores = obj[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        if confidence > confidence_threshold:
            # Calculate bounding box coordinates
            center_x, center_y, box_width, box_height = map(int, obj[0:4] * [width, height, width, height])
            x, y = center_x - box_width // 2, center_y - box_height // 2

            # Ensure class_id is within bounds of class_labels
            if class_id < len(class_labels):
                label = f'{class_labels[class_id]}: {confidence:.2f}'
            else:
                label = f'Unknown: {confidence:.2f}'

            # Draw bounding box and label
            cv2.rectangle(image, (x, y), (x + box_width, y + box_height), (0, 255, 0), 2)
            cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Define the scale factor
scale_factor = 0.8

# Calculate the new dimensions for the scaled image
new_width = int(image.shape[1] * scale_factor)
new_height = int(image.shape[0] * scale_factor)

# Resize the image with bounding boxes
image_resized = cv2.resize(image, (new_width, new_height))

# Display the scaled image with bounding boxes
cv2.imshow('Scaled Object Detection', image_resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
import cv2
import numpy as np
import os

cv2.dnn.DNN_BACKEND_CUDA = cv2.dnn.DNN_BACKEND_DEFAULT
cv2.dnn.DNN_TARGET_CUDA = cv2.dnn.DNN_TARGET_CUDA_FP16

# Load YOLOv3 model
net = cv2.dnn.readNet('cfg/yolov3.cfg', 'model/yolov3.weights')

# Define class labels and confidence threshold
class_labels = [ 'person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
                 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
                 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 
                 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 
                 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 
                 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 
                 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 
                 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 
                 'teddy bear', 'hair drier', 'toothbrush'
                ]
confidence_threshold = 0.6

# Define the scale factor (e.g., 0.5 for 50% scaling)
scale_factor = 0.8

# Get a list of video files in the "videos" folder
video_folder = "videos"
video_files = [os.path.join(video_folder, filename) for filename in os.listdir(video_folder) if filename.endswith(".mp4")]
current_video_index = 0

while current_video_index < len(video_files):
    # Load the current video
    video_file = video_files[current_video_index]
    cap = cv2.VideoCapture(video_file)
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_file}")
        current_video_index += 1
        continue

    while True:
        ret, frame = cap.read()
        if not ret:
            # End of video, move to the next video
            break

        # Resize the frame with bounding boxes
        new_width = int(frame.shape[1] * scale_factor)
        new_height = int(frame.shape[0] * scale_factor)
        frame_resized = cv2.resize(frame, (new_width, new_height))

        # Prepare the frame for YOLO model
        blob = cv2.dnn.blobFromImage(frame_resized, 1/255.0, (416, 416), swapRB=True, crop=False)
        net.setInput(blob)

        # Get the names of the output layers
        output_layers = net.getUnconnectedOutLayersNames()

        # Forward pass to get the detected objects
        detections = net.forward(output_layers)

        # Process detections and draw bounding boxes
        for detection in detections:
            for obj in detection:
                scores = obj[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]

                if confidence > confidence_threshold:
                    # Calculate bounding box coordinates
                    center_x, center_y, box_width, box_height = map(int, obj[0:4] * [new_width, new_height, new_width, new_height])
                    x, y = center_x - box_width // 2, center_y - box_height // 2

                    # Draw bounding box and label
                    cv2.rectangle(frame_resized, (x, y), (x + box_width, y + box_height), (0, 255, 0), 2)
                    label = f'{class_labels[class_id]}: {confidence:.2f}'
                    cv2.putText(frame_resized, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Display the scaled frame with bounding boxes
        cv2.imshow('Object Detection', frame_resized)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # Move to the next video
    current_video_index += 1

# End of all videos in the folder
print("All videos in the folder have been played.")

All videos in the folder have been played.
